In [1]:
from polyglot.downloader import downloader
downloader.download("embeddings2.en")
downloader.download("embeddings2.hi")
downloader.download("embeddings2.ta")
downloader.download("transliteration2.hi")
downloader.download("transliteration2.ta")

[polyglot_data] Downloading package embeddings2.en to
[polyglot_data]     /home/ubuntu/polyglot_data...
[polyglot_data]   Package embeddings2.en is already up-to-date!
[polyglot_data] Downloading package embeddings2.hi to
[polyglot_data]     /home/ubuntu/polyglot_data...
[polyglot_data]   Package embeddings2.hi is already up-to-date!
[polyglot_data] Downloading package embeddings2.ta to
[polyglot_data]     /home/ubuntu/polyglot_data...
[polyglot_data]   Package embeddings2.ta is already up-to-date!
[polyglot_data] Downloading package transliteration2.hi to
[polyglot_data]     /home/ubuntu/polyglot_data...
[polyglot_data]   Package transliteration2.hi is already up-to-date!
[polyglot_data] Downloading package transliteration2.ta to
[polyglot_data]     /home/ubuntu/polyglot_data...
[polyglot_data]   Package transliteration2.ta is already up-to-date!


True

In [2]:
from datasets import load_dataset, load_metric, Audio

common_voice_train = load_dataset("common_voice", "ta", split="train+validation")
common_voice_test = load_dataset("common_voice", "ta", split="test")

Reusing dataset common_voice (/home/ubuntu/.cache/huggingface/datasets/common_voice/ta/6.1.0/f7a9d973839b7706e9e281c19b7e512f31badf3c0fdbd21c671f3c4bf9acf3b9)
Reusing dataset common_voice (/home/ubuntu/.cache/huggingface/datasets/common_voice/ta/6.1.0/f7a9d973839b7706e9e281c19b7e512f31badf3c0fdbd21c671f3c4bf9acf3b9)


In [3]:
common_voice_train = common_voice_train.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])
common_voice_test = common_voice_test.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])

In [4]:
import re
import polyglot
from polyglot.text import Text
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'


def get_transliterations(batch):
    batch["sentence"] = Text(batch["sentence"], hint_language_code="ta")
    batch["sentence"] = (" ".join(list(batch["sentence"].transliterate("en"))))
    return batch

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).upper() + " "
    return batch

In [5]:
common_voice_train = common_voice_train.map(get_transliterations)
common_voice_test = common_voice_test.map(get_transliterations)

Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/common_voice/ta/6.1.0/f7a9d973839b7706e9e281c19b7e512f31badf3c0fdbd21c671f3c4bf9acf3b9/cache-cb78d13a18c59b24.arrow
Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/common_voice/ta/6.1.0/f7a9d973839b7706e9e281c19b7e512f31badf3c0fdbd21c671f3c4bf9acf3b9/cache-6994764b3354d72b.arrow


In [6]:
common_voice_train = common_voice_train.map(remove_special_characters)
common_voice_test = common_voice_test.map(remove_special_characters)

Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/common_voice/ta/6.1.0/f7a9d973839b7706e9e281c19b7e512f31badf3c0fdbd21c671f3c4bf9acf3b9/cache-94ecb9b3bb2b13fb.arrow
Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/common_voice/ta/6.1.0/f7a9d973839b7706e9e281c19b7e512f31badf3c0fdbd21c671f3c4bf9acf3b9/cache-5929366350c9515c.arrow


In [7]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("config/vocab.json", unk_token="<unk>", pad_token="<pad>", word_delimiter_token="|")

In [8]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [9]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [10]:
common_voice_train[0]["audio"]

{'path': '/home/ubuntu/.cache/huggingface/datasets/downloads/extracted/bbc7e5ef7e1baed3b86a20ce9e146e069a26e1bebd1feadc09a550c84ff03cf0/cv-corpus-6.1-2020-12-11/ta/clips/common_voice_ta_19171502.mp3',
 'array': array([0.        , 0.        , 0.        , ..., 0.00075823, 0.00074035,
        0.00111735], dtype=float32),
 'sampling_rate': 48000}

In [11]:
common_voice_train = common_voice_train.cast_column("audio", Audio(sampling_rate=16_000))
common_voice_test = common_voice_test.cast_column("audio", Audio(sampling_rate=16_000))

In [12]:
common_voice_train[0]["audio"]

{'path': '/home/ubuntu/.cache/huggingface/datasets/downloads/extracted/bbc7e5ef7e1baed3b86a20ce9e146e069a26e1bebd1feadc09a550c84ff03cf0/cv-corpus-6.1-2020-12-11/ta/clips/common_voice_ta_19171502.mp3',
 'array': array([0.        , 0.        , 0.        , ..., 0.00103243, 0.00046855,
        0.00098617], dtype=float32),
 'sampling_rate': 16000}

In [13]:
import random
rand_int = random.randint(0, len(common_voice_train)-1)

print("Target text:", common_voice_train[rand_int]["sentence"])
print("Input array shape:", common_voice_train[rand_int]["audio"]["array"].shape)
print("Sampling rate:", common_voice_train[rand_int]["audio"]["sampling_rate"])

Target text: VERNRI EDDUTHIKU MURNRILUME SENRU 
Input array shape: (56832,)
Sampling rate: 16000


In [14]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch

In [15]:
common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train.column_names)
common_voice_test = common_voice_test.map(prepare_dataset, remove_columns=common_voice_test.column_names)

  0%|          | 0/3788 [00:00<?, ?ex/s]

Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/common_voice/ta/6.1.0/f7a9d973839b7706e9e281c19b7e512f31badf3c0fdbd21c671f3c4bf9acf3b9/cache-af227e453f83ffbe.arrow


In [16]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [17]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [18]:
wer_metric = load_metric("wer")

In [19]:
import numpy as np
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [20]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53", 
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

Some weights of the model checkpoint at facebook/wav2vec2-large-xlsr-53 were not used when initializing Wav2Vec2ForCTC: ['project_q.weight', 'project_q.bias', 'quantizer.codevectors', 'quantizer.weight_proj.weight', 'quantizer.weight_proj.bias', 'project_hid.bias', 'project_hid.weight']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.weight', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to u

In [21]:
model.freeze_feature_extractor()
model.gradient_checkpointing_enable()

In [22]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="wav2vec2-large-rbg-tamil",
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=50,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
)

In [23]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=common_voice_train,
    eval_dataset=common_voice_test,
    tokenizer=processor.feature_extractor,
)

Using amp fp16 backend


In [ ]:
trainer.train()

***** Running training *****
  Num examples = 3788
  Num Epochs = 50
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 2
  Total optimization steps = 5900
/home/ubuntu/environments/t-seed/lib/python3.8/site-packages/transformers/feature_extraction_utils.py:158: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  tensor = as_tensor(value)
/home/ubuntu/environments/t-seed/lib/python3.8/site-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:882: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep th

Step,Training Loss,Validation Loss,Wer
100,4.316100,3.703673,1.000000
200,2.972100,3.014820,1.000000
300,2.911600,2.948166,1.000000
400,2.850400,2.874598,1.000000


***** Running Evaluation *****
  Num examples = 1781
  Batch size = 8
Saving model checkpoint to wav2vec2-large-rbg-tamil/checkpoint-100
Configuration saved in wav2vec2-large-rbg-tamil/checkpoint-100/config.json
Model weights saved in wav2vec2-large-rbg-tamil/checkpoint-100/pytorch_model.bin
Configuration saved in wav2vec2-large-rbg-tamil/checkpoint-100/preprocessor_config.json
/home/ubuntu/environments/t-seed/lib/python3.8/site-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:882: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  return (input_length - kernel_size) // stride + 1
***** Running Evaluation *****
  Num examples = 1781
  Batch size = 8
Savin